Data Processing

In [ ]:

import pandas as pd
df = pd.read_csv('/work/final_forreal.csv')

home_ohe = pd.get_dummies(df['homeType'])
home_ohe
df = pd.concat([df, home_ohe], axis = 1)
df

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
#columns=['Lead', 'Address', 'city', 'cityId', 'county', 'countyFIPS', 'homeType', 'livingArea', 'livingAreaValue', 'parcelId', 'price', 'regionString', 'state', 'stateId', 'zipcode']

X = df.drop(columns=['Lead', 'Address', 'city', 'cityId', 'county', 'countyFIPS', 'homeType', 'livingArea', 'livingAreaValue', 'parcelId', 'price', 'regionString', 'state', 'stateId', 'zipcode'])
y = df['Lead']

# Replace NaN values with the mean of the non-missing values
imputer = SimpleImputer(strategy="mean")
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Use SMOTE to oversample the positive class to 30,000 samples
smote = SMOTE(sampling_strategy={1: 50000}, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

Model

In [ ]:
# Define the binary classification neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=8),  # Change to 8 features
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with class weights, accuracy as the metric, and a learning rate of 0.00001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy', metrics=[auroc_metric])

# Train the model with class weights
model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=64, validation_split=0.2, class_weight=class_weights)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert probabilities to binary predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate confusion matrix
confusion = confusion_matrix(y_test, y_pred_binary)

# Calculate false positive rate, false negative rate, and AUROC
tn, fp, fn, tp = confusion.ravel()
false_positive_rate = fp / (fp + tn)
false_negative_rate = fn / (fn + tp)

# Print the results
print(f"Accuracy: {accuracy_score(y_test, y_pred_binary)}")
print(f"False Positive Rate: {false_positive_rate}")
print(f"False Negative Rate: {false_negative_rate}")